<a href="https://colab.research.google.com/github/IbnuSyifa/CapstoneProject/blob/main/Capstone_Ibnu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community

!pip install replicate

In [ ]:
from langchain_community.llms import Replicate
import os
import pandas as pd
from google.colab import files
from google.colab import userdata

# Set the API token

api_token = userdata.get('api_token')

os.environ["REPLICATE_API_TOKEN"] = api_token

# Model setup

model = "ibm-granite/granite-3.3-8b-instruct"

output = Replicate(model=model,replicate_api_token=api_token,)

# Upload file dari perangkat
uploaded = files.upload()

# Baca file CSV (asumsikan hanya satu file diunggah)
for filename in uploaded.keys():
    print(f"Data dari {filename} berhasil dimuat:")
    df = pd.read_csv(filename)
    print(df.head())

In [ ]:
import pandas as pd

if 'comment' not in df.columns:
    raise ValueError("Kolom 'review' tidak ditemukan dalam dataset.")


user_comment = df['comment'].dropna().tolist()


comment_text = "\n".join([f"Comment {i+1}: {comment}" for i, comment in enumerate(user_comment)])

prompt = f"""
Classify the {comment_text} as 'Positive', 'Negative', or 'Neutral'. Give the reasons why.
Show the results in a table format with table header: Comment Number, Classification, Comment, and Reason.
{comment_text}
"""

response = output.invoke(prompt)

print("Granite Model Response:\n")
print(response)


In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

df = pd.read_csv(filename)


if 'comment' not in df.columns:
    raise ValueError("Kolom 'comment' tidak ditemukan dalam dataset.")

# === Gabungkan semua teks review menjadi satu string ===
text = " ".join(comment_text for comment in df['comment'].dropna().astype(str))

# === Membuat WordCloud ===
wordcloud = WordCloud(width=800, height=400, background_color='white',
                      max_words=200, colormap='viridis').generate(text)

# === Tampilkan WordCloud ===
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud from Comment", fontsize=16)
plt.show()


In [ ]:
!pip install ace_tools

In [ ]:
import re
from collections import Counter

# Gabungkan semua komentar jadi satu string, lowercase, hapus URL/mention/punctuation
text = " ".join(df['comment'].dropna().astype(str)).lower()
text = re.sub(r'http\S+|@\S+', '', text)
text = re.sub(r'[^\w\s]', '', text)

# Tokenisasi & filter stopwords sederhana
words = text.split()
stopwords = {
    'yang','ini','dan','di','ke','dengan','untuk','gak','ga',
    'aja','banget','ya','lho','nya','itu','the','make','america',
    'great','again','all','yg','dari','jadi','ada'
}
filtered = [w for w in words if w not in stopwords and len(w) > 1]

# Hitung frekuensi & ambil top 10
common = Counter(filtered).most_common(10)
df_common = pd.DataFrame(common, columns=['Word', 'Count'])

# Tampilkan hasil di Colab
from IPython.display import display
print("Top 10 Most Frequent Words in Comments:")
display(df_common)

In [ ]:
print(comment_text)